# TP2: Planificación de Conexión de Oficinas de Atención a Internet

Integrantes: Micaela Oliva, Camila Bernardez, Sol Valeggia, Juan Castore

## Contexto
Estamos en el área de planificación de una empresa de cobros minoristas, y tenemos la responsabilidad de conectar a Internet 56 oficinas de atención al cliente, distribuidas en el micro y macrocentro de la ciudad. La empresa de telefonía nos ha propuesto hasta 10 centrales operativas que pueden ser utilizadas para abastecer estas oficinas, sujetas a las siguientes restricciones:

### Restricciones
1. Cada oficina debe estar conectada con una central operativa.
2. Cada oficina tiene una demanda de operaciones por hora, y cada central operativa tiene una capacidad máxima de 15.000 operaciones por hora. Esto implica que la suma de las operaciones por hora de las oficinas conectadas a cada central no puede superar las 15.000 operaciones.

## Objetivo
El objetivo de la planificación es determinar:
1. Qué centrales operativas utilizar.
2. Qué central operativa debe atender a cada oficina.

Para cumplir con estas restricciones y minimizar el costo total, considerando:

- **Costo de apertura**: USD 5.700 por cada central operativa utilizada.
- **Costo de cableado**: Cada oficina se conecta a una central mediante un cable directo cuya longitud es $d_{ij}$ en metros entre la oficina $i $ y la central $j$. El cable tiene un costo de USD 17 por cada 1.000 metros.


### 1. Modelo de Programación Lineal Entera
   - **Variables de Decisión**: 
     - $x_{ij}$: Variable binaria que indica si la oficina $i$ está conectada a la central $j$.
     - $y_j$: Variable binaria que indica si la central $j$ está en operación.
   - **Función Objetivo**: Minimizar el costo total, que se compone del costo de apertura de las centrales y el costo de cableado.
   - **Restricciones**: 
     - Cada oficina debe estar conectada a una y solo una central operativa.
     - La suma de operaciones de las oficinas conectadas a una central no puede exceder su capacidad máxima.
     - Solo pueden conectarse oficinas a centrales que estén en operación.



### 2. Análisis de Restricciones Adicionales
   - Implementar una nueva restricción en el modelo donde el número de oficinas conectadas a cada central no supere las 10.
   - Analizar el impacto de esta restricción en los resultados del modelo, especialmente en el costo total y en la distribución de las conexiones.


### 3. Determinación de la Capacidad Mínima
   - Incrementar gradualmente la capacidad máxima permitida de operaciones por hora para cada central y evaluar la factibilidad del modelo.
   - Determinar la capacidad mínima que permite una asignación factible para todas las oficinas.


### 4. Evaluación del Tiempo de Resolución
   - Generar instancias aleatorias variando el número de oficinas y centrales operativas.
   - Medir el tiempo de resolución del solver y determinar el tamaño de instancia que marca el límite de tiempos de resolución aceptables.


In [2]:
#Descargo datos
import pandas as pd
import numpy as np
import time
from ortools.linear_solver import pywraplp

#Llamamos al solver
solver = pywraplp.Solver.CreateSolver('SCIP') # Usamos SCIP

#Parámetros

# Leer el archivo con separación por espacio
C = pd.read_csv("centrales.txt", sep=" ", header = None)
C.columns = ['id', 'lat', 'long']
D = pd.read_csv("distancias.txt", sep="\t", header = None)
D = D.dropna(axis = 1, how='all')
D.columns = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
O = pd.read_csv("oficinas.txt", sep=" ", header = None)
O.columns = ['id', 'demanda', 'lat', 'long']


CAP = 15000 #Capacidad max que tiene cada central
F = 5700    #Costo fijo por abrir una central
CABLE = 0.017 #Costo por metro de cable

#Indices
I = list(range(len(O))) #Indices de oficinas
J = list(range(len(C))) #Indices de centrales

#Variables

y = {}  # y[j] = 1 si ela central j está abierta

for j in J:
    y[j] = solver.BoolVar(f'y[{j}]')

x = {}  # x[i, j] = 1 si la oficina i se conecta a la central j

for i in I:
    for j in J:
        x[i, j] = solver.BoolVar(f'x[{i},{j}]')

#Restricciones

#1. Cada oficina solo puede estar conectada a una central
for i in I:
    solver.Add(sum(x[i, j] for j in J) == 1)  

#2. La demanda de las oficinas conectadas debe ser menor a la capacidad total de la central
for j in J:
    solver.Add(sum(O['demanda'][i] * x[i, j] for i in I) <= CAP * y[j])

#3. Una central solo debe estar abierta si tiene una oficina conectada
for i in I:
    for j in J:
        solver.Add(x[i,j] <= y[j])

#4. Binarias
#Ya implemetado que las variables x e y sean binarias

#Función objetivo

#Sumatoria de costo total de centrales abiertas
costo_centrales = solver.Sum(F * y[j] for j in J)

#Doble Sumatoria de costo de conexion entre centrales y oficinas
costo_distancias = solver.Sum(D[f"c{j}"][i] * CABLE * x[i,j] for i in I for j in J)

solver.Minimize(costo_centrales + costo_distancias)

# Resolver
start_time = time.time()
status = solver.Solve()
end_time = time.time()

# Verificar resultado
if status == pywraplp.Solver.OPTIMAL:
    print("Solución óptima encontrada.")
    print("Valores de las variables y (centrales abiertas):")
    for j in J:
        print(f"Central {j} abierta: {y[j].solution_value()}")
    
    print("\nValores de las variables x (conexiones):")
    for j in J:
        for i in I:
            if x[i, j].solution_value() == 1:
                print(f"Oficina {i} conectada a Central {j}")
    
    print("\nValor óptimo de la función objetivo (costo total):", solver.Objective().Value())
    print(f"\nTiempo de resolución: {end_time - start_time:.4f} segundos")
else:
    print("No se encontró solución óptima.")

Solución óptima encontrada.
Valores de las variables y (centrales abiertas):
Central 0 abierta: 1.0
Central 1 abierta: 1.0
Central 2 abierta: 1.0
Central 3 abierta: 0.0
Central 4 abierta: 1.0
Central 5 abierta: 0.0
Central 6 abierta: 1.0
Central 7 abierta: 0.0
Central 8 abierta: 1.0
Central 9 abierta: 0.0

Valores de las variables x (conexiones):
Oficina 14 conectada a Central 0
Oficina 45 conectada a Central 0
Oficina 46 conectada a Central 0
Oficina 47 conectada a Central 0
Oficina 48 conectada a Central 0
Oficina 49 conectada a Central 0
Oficina 50 conectada a Central 0
Oficina 51 conectada a Central 0
Oficina 52 conectada a Central 0
Oficina 53 conectada a Central 0
Oficina 0 conectada a Central 1
Oficina 1 conectada a Central 1
Oficina 2 conectada a Central 1
Oficina 13 conectada a Central 1
Oficina 15 conectada a Central 1
Oficina 28 conectada a Central 1
Oficina 29 conectada a Central 1
Oficina 41 conectada a Central 1
Oficina 42 conectada a Central 1
Oficina 43 conectada a Cent

In [7]:
#Descargo datos
import pandas as pd
import numpy as np
import time
from ortools.linear_solver import pywraplp

#Llamamos al solver
solver = pywraplp.Solver.CreateSolver('SCIP') # Usamos SCIP

#Parámetros

# Leer el archivo con separación por espacio
C = pd.read_csv("centrales.txt", sep=" ", header = None)
C.columns = ['id', 'lat', 'long']
D = pd.read_csv("distancias.txt", sep="\t", header = None)
D = D.dropna(axis = 1, how='all')
D.columns = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
O = pd.read_csv("oficinas.txt", sep=" ", header = None)
O.columns = ['id', 'demanda', 'lat', 'long']


CAP = 15000 #Capacidad max que tiene cada central
F = 5700    #Costo fijo por abrir una central
CABLE = 0.017 #Costo por metro de cable

#Indices
I = list(range(len(O))) #Indices de oficinas
J = list(range(len(C))) #Indices de centrales

#Variables

y = {}  # y[j] = 1 si ela central j está abierta

for j in J:
    y[j] = solver.BoolVar(f'y[{j}]')

x = {}  # x[i, j] = 1 si la oficina i se conecta a la central j

for i in I:
    for j in J:
        x[i, j] = solver.BoolVar(f'x[{i},{j}]')

#Restricciones

#1. Cada oficina solo puede estar conectada a una central
for i in I:
    solver.Add(sum(x[i, j] for j in J) == 1)  

#2. La demanda de las oficinas conectadas debe ser menor a la capacidad total de la central
for j in J:
    solver.Add(sum(O['demanda'][i] * x[i, j] for i in I) <= CAP * y[j])

#3. Una central solo debe estar abierta si tiene una oficina conectada
for i in I:
    for j in J:
        solver.Add(x[i,j] <= y[j])

#4. Binarias
#Ya implemetado que las variables x e y sean binarias

#5. Opcional: Que cada central tenga como maximo 10 oficinas conectadas 
for j in J:
    solver.Add(sum(x[i,j] for i in I) <= 10*y[j])


#Función objetivo

#Sumatoria de costo total de centrales abiertas
costo_centrales = solver.Sum(F * y[j] for j in J)

#Doble Sumatoria de costo de conexion entre centrales y oficinas
costo_distancias = solver.Sum(D[f"c{j}"][i] * CABLE * x[i,j] for i in I for j in J)

solver.Minimize(costo_centrales + costo_distancias)

# Resolver
start_time = time.time()
status = solver.Solve()
end_time = time.time()

# Verificar resultado
if status == pywraplp.Solver.OPTIMAL:
    print("Solución óptima encontrada.")
    print("Valores de las variables y (centrales abiertas):")
    for j in J:
        print(f"Central {j} abierta: {y[j].solution_value()}")
    
    print("\nValores de las variables x (conexiones):")
    for j in J:
        for i in I:
            if x[i, j].solution_value() == 1:
                print(f"Oficina {i} conectada a Central {j}")
    
    print("\nValor óptimo de la función objetivo (costo total):", solver.Objective().Value())
    print(f"\nTiempo de resolución: {end_time - start_time:.4f} segundos")
else:
    print("No se encontró solución óptima.")

Solución óptima encontrada.
Valores de las variables y (centrales abiertas):
Central 0 abierta: 1.0
Central 1 abierta: 1.0
Central 2 abierta: 1.0
Central 3 abierta: 0.0
Central 4 abierta: 1.0
Central 5 abierta: 0.0
Central 6 abierta: 1.0
Central 7 abierta: 0.0
Central 8 abierta: 1.0
Central 9 abierta: 0.0

Valores de las variables x (conexiones):
Oficina 14 conectada a Central 0
Oficina 45 conectada a Central 0
Oficina 46 conectada a Central 0
Oficina 47 conectada a Central 0
Oficina 48 conectada a Central 0
Oficina 49 conectada a Central 0
Oficina 50 conectada a Central 0
Oficina 51 conectada a Central 0
Oficina 52 conectada a Central 0
Oficina 53 conectada a Central 0
Oficina 0 conectada a Central 1
Oficina 1 conectada a Central 1
Oficina 2 conectada a Central 1
Oficina 13 conectada a Central 1
Oficina 15 conectada a Central 1
Oficina 28 conectada a Central 1
Oficina 29 conectada a Central 1
Oficina 41 conectada a Central 1
Oficina 42 conectada a Central 1
Oficina 43 conectada a Cent

## Conclusiones
Este análisis permitirá tomar decisiones informadas sobre la selección y configuración de las centrales operativas y la conexión de las oficinas, optimizando tanto el costo como la eficiencia operativa en el abastecimiento de las oficinas con servicio de Internet.
